<a href="https://colab.research.google.com/github/amya-singhal/Ice_skating_predictive_model/blob/main/Multiplicative_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import csv
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from itertools import combinations
from scipy.stats import kendalltau, norm
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
from matplotlib import rc
import matplotlib as mpl
import seaborn as sns

plt.style.use('seaborn')
rc('text', usetex=False)
%matplotlib inline

In [65]:
!pip install utils
%load_ext autoreload
%aimport utils
%autoreload 1
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
def add_data(skaters_list, year, comp):
  new = []
  for i in skaters_list:
    new.append(i + [year, comp])
  return new

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
skaters_list = ['Kailani Craine', 'Olga Mikutina', 'Ekaterina Ryabova', 'Viktoriia Safonova', 'Loena Hendrickx', 'Alexandra Feigin', 'Madeline Schizas', 'Yi Zhu', 'Eliska Brezinova', 'Eva-Lotta Kiibus', 'Jenni Saarinen', 'Anastasiia Gubanova', 'Nicole Schott', 'Natasha McKay', 'Lara Naki Gutmann*', 'Wakaba Higuchi', 'Mana Kawabe', 'Kaori Sakamoto', 'Lindsay van Zundert', 'Ekaterina Kurakova', 'Anna Shcherbakova', 'Alexandra Trusova', 'Kamila Valieva' 'Yelim Kim', 'You Young', 'Josefin Taljegard', '	Alexia Paganini', 'Anastasiia Shabotova', 'Mariah Bell', 'Karen Chen', 'Alysa Liu']
# changed yelim kim and eliska brezinova
skaters_country_dict = {'Olga Mikutina': 'Austria', 'Ekaterina Ryabova': 'Azerbaijan', 'Loena Hendrickx': 'Belgium', 'Alexandra Feigin': 'Bulgaria', 'Madeline Schizas': 'Canada', 'TBD': 'Finland', 'Eliška Březinová': 'Czech Republic', 'Nicole Schott': 'Germany', 'Natasha McKay': 'Great Britain', 'Lara Naki Gutmann*': 'Italy', 'Wakaba Higuchi': 'Japan', 'Mana Kawabe': 'Japan', 'Kaori Sakamoto': 'Japan', 'Lindsay van Zundert': 'Netherlands', 'Ekaterina Kurakova': 'Poland', 'Kim Ye-lim': 'South Korea', 'You Young': 'South Korea', 'Mariah Bell': 'United States', 'Karen Chen': 'United States', 'Alysa Liu': 'United States'}

In [69]:
list_1=[['Alena KOSTORNAIA', '240.81'], ['Anna SHCHERBAKOVA', '237.76'], ['Alexandra TRUSOVA', '225.34'], ['Alexia PAGANINI', '192.88'], ['Emmi PELTONEN', '181.79'], ['Ekaterina RYABOVA', '181.49'], ['Eva Lotta KIIBUS', '181.24'], ['Alessia TORNAGHI', '172.17'], ['Mae Berenice MEITE', '172.08'], ['Ekaterina KURAKOVA', '170.24'], ['Maia MAZZARA', '170.06'], ['Linnea CEDER', '166.16'], ['Nicole SCHOTT', '162.26'], ['Viktoriia SAFONOVA', '159.91'], ['Alina URUSHADZE', '154.81'], ['Lea SERNA', '154.73'], ['Alexandra FEIGIN', '154.43'], ['Anita OSTLUND', '152.91'], ['Yasmine Kimiko YAMADA', '152.62'], ['Dasa GRM', '150.90'], ['Ivett TOTH', '150.36'], ['Eliska BREZINOVA', '145.35'], ['Natasha McKAY', '142.14'], ['Olga MIKUTINA', '130.15'], ['Nelli IOFFE', '51.70'], ['Aleksandra GOLOVKINA', '50.88'], ['Anastasiya GALUSTYAN', '50.08'], ['Valentina MATOS', '49.02'], ['Ema DOBOSZOVA', '46.27'], ['Angelina KUCHVALSKA', '45.09'], ['Antonina DUBININA', '43.62'], ['Sinem KUYUCU', '43.16'], ['Jenni SAARINEN', '42.61'], ['Anastasia GOZHVA', '40.49'], ['Niki WORIES', '38.19'], ['Klara STEPANOVA', '37.83'], ['Hana CVIJANOVIC', '36.22']]


list_2=[['Kaetlyn OSMOND', '223.23'], ['Wakaba HIGUCHI', '210.90'], ['Satoko MIYAHARA', '210.08'], ['Carolina KOSTNER', '208.88'], ['Alina ZAGITOVA', '207.72'], ['Bradie TENNELL', '199.89'], ['Gabrielle DALEMAN', '196.72'], ['Maria SOTSKOVA', '196.61'], ['Loena HENDRICKX', '192.31'], ['Mirai NAGASU', '187.52'], ['Elizabet TURSYNBAEVA', '186.85'], ['Mariah BELL', '174.40'], ['Nicole SCHOTT', '174.13'], ['Laurine LECAVELIER', '173.23'], ['Hanul KIM', '170.68'], ['Viveca LINDFORS', '166.23'], ['Kailani CRAINE', '154.41'], ['Eliska BREZINOVA', '153.14'], ['Stanislava KONSTANTINOVA', '153.03'], ['Alexia PAGANINI', '149.66'], ['Elisabetta LECCARDI', '149.17'], ['Dasa GRM', '144.51'], ['Ivett TOTH', '136.87']]


list_3=[['Alina ZAGITOVA', '238.24'], ['Evgenia MEDVEDEVA', '232.86'], ['Carolina KOSTNER', '204.25'], ['Maria SOTSKOVA', '200.81'], ['Loena HENDRICKX', '176.91'], ['Nicole RAJICOVA', '171.90'], ['Alexia PAGANINI', '161.62'], ['Mae Berenice MEITE', '159.70'], ['Emmi PELTONEN', '159.48'], ['Nicole SCHOTT', '157.84'], ['Laurine LECAVELIER', '154.11'], ['Eliska BREZINOVA', '149.69'], ['Ivett TOTH', '148.98'], ['Viveca LINDFORS', '147.89'], ['Micol CRISTINI', '147.80'], ['Lea Johanna DASTICH', '146.82'], ['Anita ÖSTLUND', '145.14'], ['Anne Line GJERSEM', '142.68'], ['Giada RUSSO', '142.38'], ['Dasa GRM', '137.31'], ['Pernille SORENSEN', '133.94'], ['Elzbieta KROPA', '133.87'], ['Anna KHNYCHENKOVA', '132.70'], ['Silvia HUGEC', '123.45']]


list_4=[['Satoko MIYAHARA', '219.71'], ['Kaori SAKAMOTO', '213.90'], ['Sofia SAMODUROVA', '198.70'], ['Bradie TENNELL', '192.89'], ['Laurine LECAVELIER', '172.41'], ['Megan WESSENBERG', '170.33'], ['Polina TSURSKAYA', '159.45'], ['Marin HONDA', '158.04'], ['Alaine CHARTRAND', '155.49'], ['Starr ANDREWS', '150.56']]


list_5=[['Elizaveta TUKTAMYSHEVA', '203.32'], ['Mako YAMASHITA', '203.06'], ['Evgenia MEDVEDEVA', '197.91'], ['Mariah BELL', '190.25'], ['Elizabet TURSYNBAEVA', '185.71'], ['Wakaba HIGUCHI', '181.29'], ['Starr ANDREWS', '174.72'], ['Alaine CHARTRAND', '172.17'], ['Daria PANENKOVA', '168.54'], ['Alicia PINEAULT', '158.29'], ['Yura MATSUDA', '157.59']]


list_6=[['Alina ZAGITOVA', '215.29'], ['Stanislava KONSTANTINOVA', '197.57'], ['Kaori SAKAMOTO', '197.42'], ['Yuna SHIRAIWA', '191.46'], ['Loena HENDRICKX', '191.22'], ['Daria PANENKOVA', '161.48'], ['Hanul KIM', '160.15'], ['Viveca LINDFORS', '159.62'], ['Emmi PELTONEN', '158.72'], ['Rika HONGO', '156.59'], ['Angela WANG', '149.57']]


list_7=[['Rika KIHIRA', '224.31'], ['Satoko MIYAHARA', '219.47'], ['Elizaveta TUKTAMYSHEVA', '219.02'], ['Mai MIHARA', '204.20'], ['Mariah BELL', '198.96'], ['Eunsoo LIM', '196.31'], ['Alena LEONOVA', '194.15'], ['Courtney HICKS', '178.07'], ['Maria SOTSKOVA', '176.99'], ['Mae Berenice MEITE', '162.58'], ['Angela WANG', '159.36'], ['Kailani CRAINE', '154.22']]


list_8=[['Alina ZAGITOVA', '222.95'], ['Sofia SAMODUROVA', '198.01'], ['Eunsoo LIM', '185.67'], ['Alexia PAGANINI', '182.50'], ['Yuna SHIRAIWA', '180.93'], ['Elizabet TURSYNBAEVA', '180.45'], ['Mako YAMASHITA', '161.22'], ['Polina TSURSKAYA', '149.45'], ['Yura MATSUDA', '137.99'], ['Gracie GOLD', '\xa0']]


list_9=[['Rika KIHIRA', '205.92'], ['Mai MIHARA', '202.81'], ['Bradie TENNELL', '197.78'], ['Evgenia MEDVEDEVA', '192.81'], ['Stanislava KONSTANTINOVA', '189.67'], ['Marin HONDA', '188.61'], ['Maria SOTSKOVA', '177.59'], ['Mae Berenice MEITE', '168.02'], ['Laurine LECAVELIER', '157.24'], ['Alexia PAGANINI', '156.51'], ['Lea SERNA', '149.49'], ['Matilda ALGOTSSON', '146.35']]


list_10=[['Rika KIHIRA', '233.12'], ['Alina ZAGITOVA', '226.53'], ['Elizaveta TUKTAMYSHEVA', '215.32'], ['Kaori SAKAMOTO', '211.68'], ['Sofia SAMODUROVA', '204.33'], ['Satoko MIYAHARA', '201.31']]


list_11=[['Rika KIHIRA', '205.92'], ['Mai MIHARA', '202.81'], ['Bradie TENNELL', '197.78'], ['Evgenia MEDVEDEVA', '192.81'], ['Stanislava KONSTANTINOVA', '189.67'], ['Marin HONDA', '188.61'], ['Maria SOTSKOVA', '177.59'], ['Mae Berenice MEITE', '168.02'], ['Laurine LECAVELIER', '157.24'], ['Alexia PAGANINI', '156.51'], ['Lea SERNA', '149.49'], ['Matilda ALGOTSSON', '146.35']]


list_12=[['Satoko MIYAHARA', '219.71'], ['Kaori SAKAMOTO', '213.90'], ['Sofia SAMODUROVA', '198.70'], ['Bradie TENNELL', '192.89'], ['Laurine LECAVELIER', '172.41'], ['Megan WESSENBERG', '170.33'], ['Polina TSURSKAYA', '159.45'], ['Marin HONDA', '158.04'], ['Alaine CHARTRAND', '155.49'], ['Starr ANDREWS', '150.56']]


list_13=[['Alexandra TRUSOVA', '241.02'], ['Rika KIHIRA', '230.33'], ['Young YOU', '217.49'], ['Bradie TENNELL', '211.31'], ['Evgenia MEDVEDEVA', '209.62'], ['Marin HONDA', '179.26'], ['Yelim KIM', '176.93'], ['Serafima SAKHANOVICH', '175.97'], ['Alexia PAGANINI', '166.20'], ['Gabrielle DALEMAN', '164.34'], ['Alicia PINEAULT', '161.37'], ['Veronik MALLET', '147.79']]


list_14=[['Alena KOSTORNAIA', '240.00'], ['Rika KIHIRA', '231.84'], ['Alina ZAGITOVA', '217.99'], ['Yuhana YOKOI', '189.54'], ['Mako YAMASHITA', '189.25'], ['Sofia SAMODUROVA', '183.27'], ['Eunsoo LIM', '172.47'], ['Starr ANDREWS', '166.72'], ['Karen CHEN', '165.70'], ['Kailani CRAINE', '165.46'], ['Mae Berenice MEITE', '159.98'], ['Megan WESSENBERG', '131.73']]


list_15=[['Alexandra TRUSOVA', '234.47'], ['Evgenia MEDVEDEVA', '225.76'], ['Mariah BELL', '205.67'], ['Satoko MIYAHARA', '192.42'], ['Ekaterina RYABOVA', '187.77'], ['Yuhana YOKOI', '182.68'], ['Alexia PAGANINI', '179.69'], ['Hongyi CHEN', '175.77'], ['Nicole SCHOTT', '172.08'], ['Yuna SHIRAIWA', '170.03'], ['Stanislava KONSTANTINOVA', '156.94'], ['Emmi PELTONEN', '152.50']]


list_16=[['Alena KOSTORNAIA', '236.00'], ['Alina ZAGITOVA', '216.06'], ['Mariah BELL', '212.89'], ['Kaori SAKAMOTO', '199.24'], ['Starr ANDREWS', '180.54'], ['Wakaba HIGUCHI', '174.12'], ['Nicole SCHOTT', '166.89'], ['Lea SERNA', '166.02'], ['Yuna SHIRAIWA', '161.71'], ['Mae Berenice MEITE', '157.45'], ['Maria SOTSKOVA', '144.89'], ['Laurine LECAVELIER', '']]


list_17=[['Alena KOSTORNAIA', '247.59'], ['Anna SHCHERBAKOVA', '240.92'], ['Alexandra TRUSOVA', '233.18'], ['Rika KIHIRA', '216.47'], ['Bradie TENNELL', '212.18'], ['Alina ZAGITOVA', '205.23']]


list_18=[['Alina ZAGITOVA', '237.50'], ['Elizabet TURSYNBAEVA', '224.76'], ['Evgenia MEDVEDEVA', '223.80'], ['Rika KIHIRA', '223.49'], ['Kaori SAKAMOTO', '222.83'], ['Satoko MIYAHARA', '215.95'], ['Bradie TENNELL', '213.47'], ['Sofia SAMODUROVA', '208.58'], ['Mariah BELL', '208.07'], ['Eunsoo LIM', '205.57'], ['Gabrielle DALEMAN', '192.67'], ['Loena HENDRICKX', '186.29'], ['Ekaterina RYABOVA', '179.88'], ['Yi Christy LEUNG', '177.22'], ['Laurine LECAVELIER', '170.59'], ['Nicole SCHOTT', '170.56'], ['Alexandra FEIGIN', '165.31'], ['Dasa GRM', '161.16'], ['Hongyi CHEN', '157.59'], ['Eliska BREZINOVA', '153.45'], ['Natasha MCKAY', '151.56'], ['Eva Lotta KIIBUS', '149.99'], ['Alaine CHARTRAND', '148.97'], ['Isadora WILLIAMS', '143.22'], ['Ivett TOTH', '54.87'], ['Pernille SORENSEN', '54.36'], ['Marina PIREDDA', '53.27'], ['Emmi PELTONEN', '53.22'], ['Julia SAUTER', '53.11'], ['Anita OSTLUND', '53.07'], ['Roberta RODEGHIERO', '51.50'], ['Nicole RAJICOVA', '51.22'], ['Alexia PAGANINI', '50.51'], ['Valentina MATOS', '50.25'], ['Aurora COTOP', '48.83'], ['Kailani CRAINE', '48.82'], ['Sophia SCHALLER', '48.72'], ['Elzbieta KROPA', '47.95'], ['Anastasiya GALUSTYAN', '47.75'], ['Kyarha VAN TIEL', '41.85']]


list_19=[['Rika KIHIRA', '221.99'], ['Elizabet TURSYNBAEVA', '207.46'], ['Mai MIHARA', '207.12'], ['Kaori SAKAMOTO', '206.79'], ['Bradie TENNELL', '202.07'], ['Mariah BELL', '193.94'], ['Eunsoo LIM', '191.85'], ['Yelim KIM', '187.93'], ['Veronik MALLET', '170.46'], ['Larkyn AUSTMAN', '165.21'], ['Ting CUI', '164.84'], ['Yi Christy LEUNG', '164.79'], ['Hanul KIM', '162.48'], ['Hongyi CHEN', '150.50'], ['Kailani CRAINE', '149.52'], ['Alaine CHARTRAND', '147.54'], ['Isadora WILLIAMS', '138.26'], ['Alisson Krystle PERTICHETO', '136.97'], ['Amy LIN', '134.56'], ['Andrea MONTESINOS CANTU', '124.51'], ['Joanna SO', '117.82'], ['Brooklee HAN', '\xa0']]


list_20=[['Sofia SAMODUROVA', '213.84'], ['Alina ZAGITOVA', '198.34'], ['Viveca LINDFORS', '194.40'], ['Stanislava KONSTANTINOVA', '189.72'], ['Laurine LECAVELIER', '180.05'], ['Alexia PAGANINI', '179.90'], ['Mae Berenice MEITE', '177.10'], ['Emmi PELTONEN', '170.03'], ['Nicole RAJICOVA', '169.03'], ['Eliska BREZINOVA', '166.77'], ['Alexandra FEIGIN', '164.20'], ['Ekaterina RYABOVA', '163.17'], ['Ivett TOTH', '160.83'], ['Julia SAUTER', '153.15'], ['Yasmine Kimiko YAMADA', '151.12'], ['Nicole SCHOTT', '149.26'], ['Dasa GRM', '147.29'], ['Anita ÖSTLUND', '144.66'], ['Lucrezia GENNARO', '143.10'], ['Natasha MCKAY', '140.08'], ['Nathalie WEINZIERL', '134.58'], ['Anastasiya GALUSTYAN', '132.63'], ['Pernille SORENSEN', '131.78'], ['Antonina DUBININA', '120.25'], ['Elzbieta GABRYSZAK', '45.77'], ['Sophia SCHALLER', '44.20'], ['Gerli LIINAMÄE', '44.08'], ['Kyarha VAN TIEL', '44.00'], ['Lara Naki GUTMANN', '43.96'], ['Silvia HUGEC', '43.33'], ['Valentina MATOS', '42.86'], ['Paulina RAMANAUSKAITE', '42.31'], ['Camilla GJERSEM', '39.81'], ['Hana CVIJANOVIC', '38.00'], ['Elizabete JUBKANE', '37.75'], ['Anastasia GOZHVA', '35.51']]


list_21=[['Alena KOSTORNAIA', '240.81'], ['Anna SHCHERBAKOVA', '237.76'], ['Alexandra TRUSOVA', '225.34'], ['Alexia PAGANINI', '192.88'], ['Emmi PELTONEN', '181.79'], ['Ekaterina RYABOVA', '181.49'], ['Eva Lotta KIIBUS', '181.24'], ['Alessia TORNAGHI', '172.17'], ['Mae Berenice MEITE', '172.08'], ['Ekaterina KURAKOVA', '170.24'], ['Maia MAZZARA', '170.06'], ['Linnea CEDER', '166.16'], ['Nicole SCHOTT', '162.26'], ['Viktoriia SAFONOVA', '159.91'], ['Alina URUSHADZE', '154.81'], ['Lea SERNA', '154.73'], ['Alexandra FEIGIN', '154.43'], ['Anita OSTLUND', '152.91'], ['Yasmine Kimiko YAMADA', '152.62'], ['Dasa GRM', '150.90'], ['Ivett TOTH', '150.36'], ['Eliska BREZINOVA', '145.35'], ['Natasha McKAY', '142.14'], ['Olga MIKUTINA', '130.15'], ['Nelli IOFFE', '51.70'], ['Aleksandra GOLOVKINA', '50.88'], ['Anastasiya GALUSTYAN', '50.08'], ['Valentina MATOS', '49.02'], ['Ema DOBOSZOVA', '46.27'], ['Angelina KUCHVALSKA', '45.09'], ['Antonina DUBININA', '43.62'], ['Sinem KUYUCU', '43.16'], ['Jenni SAARINEN', '42.61'], ['Anastasia GOZHVA', '40.49'], ['Niki WORIES', '38.19'], ['Klara STEPANOVA', '37.83'], ['Hana CVIJANOVIC', '36.22']]


list_22=[['Rika KIHIRA', '232.34'], ['Young YOU', '223.23'], ['Bradie TENNELL', '222.97'], ['Wakaba HIGUCHI', '207.46'], ['Kaori SAKAMOTO', '202.79'], ['Yelim KIM', '202.76'], ['Karen CHEN', '201.06'], ['Eunsoo LIM', '200.59'], ['Amber GLENN', '190.83'], ['Alicia PINEAULT', '173.55'], ['Hongyi CHEN', '167.26'], ['Kailani CRAINE', '161.15'], ['Yi ZHU', '155.41'], ['Alison SCHUMACHER', '150.73'], ['Emily BAUSBACK', '147.23'], ['Andrea MONTESINOS CANTU', '135.24'], ['Jenny SHYU', '134.80'], ['Alisson Krystle PERTICHETO', '129.99'], ['Cheuk Ka Kahlen CHEUNG', '121.29'], ['Amy LIN', '116.97'], ['Aiza MAMBEKOVA', '107.68']]


list_23=[['Anna SHCHERBAKOVA', '229.69'], ['Alena KOSTORNAIA', '221.85'], ['Wakaba HIGUCHI', '204.91'], ['Kseniia SINITSYNA', '198.76'], ['Karen CHEN', '194.00'], ['Mariah BELL', '190.79'], ['Ekaterina RYABOVA', '186.65'], ['Yeonjeong PARK', '186.11'], ['Yuhana YOKOI', '176.93'], ['Haein LEE', '171.32'], ['Lea SERNA', '170.33'], ['Starr ANDREWS', '']]


list_24=[['Kaori SAKAMOTO', '229.51'], ['Wakaba HIGUCHI', '200.98'], ['Rino MATSUIKE', '198.97'], ['Mai MIHARA', '194.73'], ['Mako YAMASHITA', '186.13'], ['Mana KAWABE', '185.70'], ['Young YOU', '181.73'], ['Yuhana YOKOI', '176.49'], ['Marin HONDA', '162.57'], ['Tomoe KAWABATA', '162.24'], ['Nana ARAKI', '162.15'], ['Chisato URAMATSU', '162.14']]


list_25=[['Mariah BELL', '212.73'], ['Bradie TENNELL', '211.07'], ['Audrey SHIN', '206.15'], ['Karen CHEN', '204.90'], ['Amber GLENN', '190.09'], ['Shan LIN', '182.11'], ['Paige RYDBERG', '178.13'], ['Starr ANDREWS', '171.70'], ['Sierra VENETTA', '170.72'], ['Pooja KALYAN', '158.95'], ['Finley HAWK', '154.25'], ['Gracie GOLD', '127.82']]


list_27=[['Elizaveta TUKTAMYSHEVA', '223.39'], ['Alena KOSTORNAIA', '220.78'], ['Anastasiia GULIAKOVA', '199.03'], ['Alexandra TRUSOVA', '198.93'], ['Elizaveta NUGUMANOVA', '191.52'], ['Eva-Lotta KIIBUS', '186.00'], ['Sofia SAMODUROVA', '184.81'], ['Viktoriia SAFONOVA', '184.57'], ['Ekaterina RYABOVA', '167.85'], ['Alina URUSHADZE', '150.68'], ['Anastasiya GALUSTYAN', '52.06']]


list_28=[['Hongyi CHEN', '186.53'], ['Angel LI', '148.33'], ['Minzhi JIN', '135.43'], ['Siyang ZHANG', '128.20'], ['Lu ZHENG', '86.48']]


list_29=[['Anna SHCHERBAKOVA', '229.69'], ['Alena KOSTORNAIA', '221.85'], ['Wakaba HIGUCHI', '204.91'], ['Kseniia SINITSYNA', '198.76'], ['Karen CHEN', '194.00'], ['Mariah BELL', '190.79'], ['Ekaterina RYABOVA', '186.65'], ['Yeonjeong PARK', '186.11'], ['Yuhana YOKOI', '176.93'], ['Haein LEE', '171.32'], ['Lea SERNA', '170.33'], ['Starr ANDREWS', '']]


list_30=[['Anna SHCHERBAKOVA', '233.17'], ['Elizaveta TUKTAMYSHEVA', '220.46'], ['Alexandra TRUSOVA', '217.20'], ['Karen CHEN', '208.63'], ['Loena HENDRICKX', '208.44'], ['Kaori SAKAMOTO', '207.80'], ['Rika KIHIRA', '205.70'], ['Olga MIKUTINA', '198.77'], ['Bradie TENNELL', '197.81'], ['Haein LEE', '193.44'], ['Yelim KIM', '191.78'], ['Ekaterina RYABOVA', '189.46'], ['Madeline SCHIZAS', '185.78'], ['Eva-Lotta KIIBUS', '181.47'], ['Josefin TALJEGARD', '178.10'], ['Lindsay van ZUNDERT', '174.50'], ['Alexandra FEIGIN', '173.52'], ['Nicole SCHOTT', '172.80'], ['Satoko MIYAHARA', '172.30'], ['Alina URUSHADZE', '169.01'], ['Hongyi CHEN', '162.79'], ['Eliska BREZINOVA', '155.14'], ['Natasha McKAY', '153.46'], ['Jenni SAARINEN', '146.54'], ['Alexia PAGANINI', '57.23'], ['Kailani CRAINE', '56.86'], ['Emily BAUSBACK', '55.74'], ['Lara Naki GUTMANN', '55.64'], ['Emmy MA', '55.63'], ['Julia LANG', '54.20'], ['Nelli IOFFE', '52.43'], ['Ekaterina KURAKOVA', '52.28'], ['Angelina KUCHVALSKA', '47.94'], ['Dasa GRM', '47.76'], ['Anastasiia ARKHIPOVA', '45.07'], ['Emilea ZINGAS', '43.20'], ['Elzbieta KROPA', '41.31'], ['Mae Berenice MEITE', ''], ['Yi Christy LEUNG', '']]


list_31=[['Kaori SAKAMOTO', '223.34'], ['Mana KAWABE', '205.44'], ['Young YOU', '203.60'], ['Alysa LIU', '202.90'], ['Eunsoo LIM', '186.68'], ['Rino MATSUIKE', '186.17'], ['Amber GLENN', '175.83'], ['Nicole SCHOTT', '172.37'], ['Seoyeong WI', '170.54'], ['Daria USACHEVA', '']]


list_32=[['Alexandra TRUSOVA', '232.37'], ['Daria USACHEVA', '217.31'], ['Young YOU', '216.97'], ['Kaori SAKAMOTO', '215.93'], ['Kseniia SINITSYNA', '205.76'], ['Amber GLENN', '201.02'], ['Satoko MIYAHARA', '200.51'], ['Yelim KIM', '199.34'], ['Ekaterina KURAKOVA', '188.60'], ['Starr ANDREWS', '177.63'], ['Yuhana YOKOI', '174.07'], ['Audrey SHIN', '160.78']]


list_33=[['Kamila VALIEVA', '265.08'], ['Elizaveta TUKTAMYSHEVA', '232.88'], ['Alena KOSTORNAIA', '214.54'], ['Mai MIHARA', '210.01'], ['Alysa LIU', '206.53'], ['Wakaba HIGUCHI', '205.27'], ['Haein LEE', '190.00'], ['Madeline SCHIZAS', '186.56'], ['Mana KAWABE', '186.52'], ['Karen CHEN', '183.41'], ['Emily BAUSBACK', '159.88'], ['Alison SCHUMACHER', '151.19']]


list_34=[['Anna SHCHERBAKOVA', '229.69'], ['Alena KOSTORNAIA', '221.85'], ['Wakaba HIGUCHI', '204.91'], ['Kseniia SINITSYNA', '198.76'], ['Karen CHEN', '194.00'], ['Mariah BELL', '190.79'], ['Ekaterina RYABOVA', '186.65'], ['Yeonjeong PARK', '186.11'], ['Yuhana YOKOI', '176.93'], ['Haein LEE', '171.32'], ['Lea SERNA', '170.33'], ['Starr ANDREWS', '']]


list_35=[['Kamila VALIEVA', '272.71'], ['Elizaveta TUKTAMYSHEVA', '229.23'], ['Maiia KHROMYKH', '219.69'], ['Mariah BELL', '210.35'], ['Loena HENDRICKX', '203.69'], ['Madeline SCHIZAS', '192.14'], ['Viktoriia SAFONOVA', '185.64'], ['Rino MATSUIKE', '184.36'], ['Ekaterina KURAKOVA', '175.64'], ['Ekaterina RYABOVA', '175.24'], ['Eva-Lotta KIIBUS', '163.11'], ['Olga MIKUTINA', '161.09']]


list_36=[['Anna SHCHERBAKOVA', '236.78'], ['Maiia KHROMYKH', '226.35'], ['Loena HENDRICKX', '219.05'], ['Mai MIHARA', '214.95'], ['Satoko MIYAHARA', '209.57'], ['Yelim KIM', '193.50'], ['Sofia SAMODUROVA', '180.59'], ['Eunsoo LIM', '179.58'], ['Yi ZHU', '171.25'], ['Nicole SCHOTT', '167.20'], ['Lara Naki GUTMANN', '158.57'], ['Lucrezia BECCARI', '148.29']]


In [70]:
list_1 = add_data(list_1, '2020', 'EC')
list_2 = add_data(list_2, '2018', 'WC')
list_3 = add_data(list_3, '2018', 'EC')
list_4 = add_data(list_4, '2018', 'SA')
list_5 = add_data(list_5, '2018', 'SC')
list_6 = add_data(list_6, '2018', 'HC')
list_7 = add_data(list_7, '2018', 'NHK')
list_8 = add_data(list_8, '2018', 'RC')
list_9 = add_data(list_9, '2018', 'F')
list_10 = add_data(list_10, '2018', 'GP')
list_11 = add_data(list_11, '2019', 'F')
list_12 = add_data(list_12, '2019', 'SA')
list_13 = add_data(list_13,"2019","SC")
list_14 = add_data(list_14,"2019","NHK")
list_15 = add_data(list_15,"2019","RC")
list_16 = add_data(list_16,"2019","F")
list_17 = add_data(list_17,"2019","GP")
list_18 = add_data(list_18,"2019","WC")
list_19 = add_data(list_19,"2019","4C")
list_20 = add_data(list_20,"2019","EC")
list_21 = add_data(list_21,"2020","EC")
list_22 = add_data(list_22,"2020","4C")
list_23 = add_data(list_23,"2021","F")
list_24 = add_data(list_24,"2020","NHK")
list_25 = add_data(list_25, "2020", "SA")
list_27 = add_data(list_27, "2020", "RC")
list_28 = add_data(list_28, "2020", "CC")
list_29 = add_data(list_29, "2020", "F")
list_30 = add_data(list_30, "2021", "WC")
list_31 = add_data(list_31, "2021", "NHK")
list_32 = add_data(list_32, "2021", "SA")
list_33 = add_data(list_33, "2021", "SC")
list_34 = add_data(list_34, "2021", "F")
list_35 = add_data(list_35, "2021", "SC")
list_36 = add_data(list_36, "2021", "I")

In [71]:
final_list= list_1+list_2+list_3+list_4+list_5+list_6+list_7+list_8+list_9+list_10+list_11+list_12+list_13+list_14+list_15+list_16+list_17+list_18+list_19+list_20+list_21+list_22+list_23+list_24+list_25+list_27+list_28+list_29+list_30+list_31+list_32+list_33+list_34+list_35+list_36


In [72]:
with open('ice_skating.csv','w') as f:
    for row in final_list:
        for x in row:
          if x == row[-1]:
            f.write(str(x))
          else:
            f.write(str(x) + ',')
        f.write('\n')

In [73]:
trimmed_list = []
with open('ice_skating.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
      if row[0].title() in skaters_list:
        trimmed_list.append(row) 

In [74]:
with open('trimmed_ice_skating.csv','w') as f:
  csvwriter = csv.writer(f, delimiter=',')
  csvwriter.writerow(['name','score','year','event'])
  csvwriter.writerows(trimmed_list)
  
  for row in trimmed_list:
      for x in row:
        if x == row[-1]:
          f.write(str(x))
        else :
          f.write(str(x) + ',')
      f.write('\n')      

In [75]:
with open('trimmed_ice_skating.csv','r') as f:
  data = f.read()

In [76]:
ice_skating_final = pd.read_csv('trimmed_ice_skating.csv')
ice_skating_final["score"] = pd.to_numeric(ice_skating_final["score"], downcast="float")

In [77]:
years_before_last = range(2018, 2021)
train_years = [2018,2019,2020]
test_years = [2021]
train_years, test_years

([2018, 2019, 2020], [2021])

In [78]:
def train_test_split(scores, train_years, test_years):
    season_scores = scores.loc[scores['event']!='WC']
    world_scores = scores.loc[scores['event']=='WC']
    
    season_train = season_scores.loc[season_scores['year'].isin(train_years)]
    world_train = world_scores.loc[world_scores['year'].isin(train_years)]
    season_test = season_scores.loc[season_scores['year'].isin(test_years)]
    world_test = world_scores.loc[world_scores['year'].isin(test_years)]    
    
    return season_scores, world_scores, season_train, world_train, season_test, world_test

In [79]:
season_scores, world_scores,season_train, world_train, season_test, world_test = train_test_split(ice_skating_final, train_years, test_years)
season_train.shape, world_train.shape, season_test.shape, world_test.shape

((142, 4), (28, 4), (68, 4), (32, 4))

In [80]:
def return_ranking(skater_scores, world_scores):
    skater_scores= skater_scores.sort_values('score',ascending= False)
    world_scores = world_scores.sort_values('score',ascending= False)
    skater_ranking = list(skater_scores.index.intersection(world_scores.index))
    world_ranking = list(world_scores.index.intersection(skater_scores.index))
    return skater_ranking, world_ranking

In [81]:
def calculate_kendall_tau(skater_ranking, world_ranking, verbose=True):
    '''
    Caluculate kendall's tau from two ranking lists of the same size
    '''
    skater_pairs = set(combinations(skater_ranking, 2))
    world_pairs = set(combinations(world_ranking, 2))
    n_pairs = len(skater_pairs)
    n_concordant_pairs = len(skater_pairs & world_pairs)
    
    if verbose:
        print(f'There are {n_concordant_pairs} concordant_pairs out of {n_pairs} pairs')
        
    tau = (2 * n_concordant_pairs - n_pairs) / n_pairs
    return tau 

In [82]:
season_avg = season_scores.groupby(['name']).mean().sort_values('score',ascending=False)
world_avg = world_scores.groupby(['name']).mean().sort_values('score',ascending=False)

season_avg.head()
season_avg.to_csv('ice_skating_final.csv')

In [83]:
avg_ranking, world_ranking = return_ranking(season_avg, world_avg)

avg_kendall_tau = calculate_kendall_tau(avg_ranking, world_ranking)
avg_kendall_tau

There are 114 concordant_pairs out of 136 pairs


0.6764705882352942

In [84]:
season_numeric_rank = list(range(len(avg_ranking)))
world_numeric_rank = [avg_ranking.index(skater) for skater in world_ranking]
kendalltau(season_numeric_rank, world_numeric_rank)

KendalltauResult(correlation=0.676470588235294, pvalue=5.18722751399025e-05)

In [85]:
score_comparison = pd.merge(season_scores, season_avg, left_on='name', right_index=True, suffixes=['', '_avg'])
score_comparison['sq_error'] = (score_comparison['score'] - score_comparison['score_avg'])**2
np.sqrt(score_comparison['sq_error'].mean())

9.721671398241694

In [86]:
dummies = pd.get_dummies(season_scores[['name', 'event']], prefix=['', ''], prefix_sep='', drop_first=True)
unique_skaters = season_scores['name'].unique()
unique_events = season_scores['event'].unique()

dummies_skater_count = len(unique_skaters) - 1
dummies_skaters = dummies.columns[:dummies_skater_count]
dummies_events = dummies.columns[dummies_skater_count:]

dropped_skater = list(set(unique_skaters) - set(dummies_skaters))[0]
dropped_event = list(set(unique_events) - set(dummies_events))[0]

In [87]:
X = dummies.values
y = season_scores['score'].values
print(y)
X = np.insert(X, obj=0, values=1, axis=1)

coefs_linear = np.linalg.inv(X.T @ X) @ (X.T @ y)
coefs_linear[:10]

[237.76 225.34 181.49 170.24 162.26 159.91 154.43 145.35 130.15  42.61
 176.91 157.84 149.69 213.9  190.25 181.29 197.42 191.22 198.96 154.22
 211.68 213.9  241.02 165.7  165.46 234.47 205.67 187.77 172.08 212.89
 199.24 174.12 166.89 240.92 233.18 206.79 193.94 149.52 166.77 164.2
 163.17 149.26 237.76 225.34 181.49 170.24 162.26 159.91 154.43 145.35
 130.15  42.61 207.46 202.79 201.06 161.15 155.41 229.69 204.91 194.
 190.79 186.65 229.51 200.98 185.7  212.73 204.9  198.93 186.   184.57
 167.85 229.69 204.91 194.   190.79 186.65 223.34 205.44 202.9  172.37
 232.37 215.93 188.6  206.53 205.27 186.56 186.52 183.41 229.69 204.91
 194.   190.79 186.65 210.35 203.69 192.14 185.64 175.64 175.24 163.11
 161.09 236.78 219.05 171.25 167.2  237.76 225.34 181.49 170.24 162.26
 159.91 154.43 145.35 130.15  42.61 176.91 157.84 149.69 213.9  190.25
 181.29 197.42 191.22 198.96 154.22 211.68 213.9  241.02 165.7  165.46
 234.47 205.67 187.77 172.08 212.89 199.24 174.12 166.89 240.92 233.18
 206.79 1

array([ 167.79056449,   60.9518312 ,   35.51979791,   68.77295196,
         11.46653397,   15.98050933,   -5.89599609,    7.39298573,
       -115.07600403,  -11.74131002])

In [88]:
linear_rmse = np.sqrt(np.mean((y - X @ coefs_linear)**2))
linear_rmse

8.593198370238037

In [89]:
lin = LinearRegression(fit_intercept=False)
lin.fit(X, y)
lin.coef_[:10], lin.intercept_

(array([ 167.79179231,   60.95104746,   35.51892172,   68.77214685,
          11.46578622,   15.97974431,   -5.89665604,    7.39219   ,
        -115.07666016,  -11.74237633]), 0.0)

In [90]:
np.sqrt(mean_squared_error(y, X @ lin.coef_))


8.593198368897761

In [91]:
baseline_score = coefs_linear[0]
skater_scores = pd.Series(coefs_linear[1:dummies_skater_count+1], index=dummies_skaters)
event_scores = pd.Series(coefs_linear[dummies_skater_count+1:], index=dummies_events)

skater_scores[dropped_skater] = 0
event_scores[dropped_event] = 0

skater_scores = pd.DataFrame(skater_scores)
skater_scores.columns = ['score']

skater_scores.sort_values('score', ascending=False, inplace=True)
event_scores.sort_values(ascending=False, inplace=True)

In [92]:
baseline_score

167.7905644912763

In [93]:
event_scores

SA      8.167908
I       7.232062
GP      3.741486
NHK     3.076479
4C      0.000000
SC     -0.267211
RC     -0.989904
F      -1.473002
EC    -10.104560
HC    -10.921148
dtype: float64

In [94]:
skater_scores = pd.DataFrame(skater_scores)


In [95]:
world_scores = world_scores.set_index('name')

In [96]:
linear_ranking, world_ranking = return_ranking(skater_scores, world_scores)

In [97]:
calculate_kendall_tau(linear_ranking, world_ranking)

There are 116 concordant_pairs out of 136 pairs


0.7058823529411765

In [98]:
coefs_loglinear = np.linalg.inv(X.T @ X) @ (X.T @ np.log(y))
coefs_loglinear[:10]

array([ 5.11931634,  0.31424062,  0.1943665 ,  0.3500189 ,  0.07028326,
        0.09247072, -0.03933827,  0.04147866, -1.30811707, -0.06768488])

In [99]:
loglinear_rmse = np.sqrt(np.mean((y - np.exp(X @ coefs_loglinear))**2))
loglinear_rmse

8.772373361332418

In [100]:
baseline_score = coefs_loglinear[0]
skater_scores = pd.Series(coefs_loglinear[1:dummies_skater_count+1], index=dummies_skaters)
event_scores = pd.Series(coefs_loglinear[dummies_skater_count+1:], index=dummies_events)

skater_scores[dropped_skater] = 0
event_scores[dropped_event] = 0

skater_scores.sort_values(ascending=False, inplace=True)
event_scores.sort_values(ascending=False, inplace=True)

baseline_score = np.exp(baseline_score)
skater_scores = np.exp(skater_scores)
event_scores = np.exp(event_scores)

In [101]:
baseline_score


167.2210087745456

In [102]:
event_scores

SA     1.042569
I      1.038242
GP     1.015736
NHK    1.015291
SC     1.000619
4C     1.000000
RC     0.997965
F      0.994696
HC     0.948873
EC     0.942603
dtype: float64

In [103]:
skater_scores


Anna SHCHERBAKOVA     1.419094
Alexandra TRUSOVA     1.369219
Kaori SAKAMOTO        1.248948
Alysa LIU             1.214541
Loena HENDRICKX       1.200557
Mariah BELL           1.188914
Wakaba HIGUCHI        1.182398
Mana KAWABE           1.138414
Karen CHEN            1.133126
Madeline SCHIZAS      1.131511
Ekaterina RYABOVA     1.096881
Ekaterina KURAKOVA    1.072812
Viktoriia SAFONOVA    1.060095
Eva-Lotta KIIBUS      1.042351
Nicole SCHOTT         1.002048
Alexandra FEIGIN      1.000000
Eliska BREZINOVA      0.961425
Yi ZHU                0.957447
Kailani CRAINE        0.934555
Olga MIKUTINA         0.869066
Jenni SAARINEN        0.270329
dtype: float64

In [104]:
lin = LinearRegression(fit_intercept=False)
lin.fit(X, np.log(y))
lin.coef_[:10], lin.intercept_

(array([ 5.11928563,  0.31426071,  0.19438958,  0.35003878,  0.07030294,
         0.09249079, -0.03932126,  0.04150001, -1.30810006, -0.06765757]), 0.0)

In [105]:
skater_scores = pd.DataFrame(skater_scores)
skater_scores.columns = ['score']

In [106]:
loglinear_ranking, world_ranking = return_ranking(skater_scores, world_scores)


In [107]:
calculate_kendall_tau(linear_ranking, world_ranking)


There are 116 concordant_pairs out of 136 pairs


0.7058823529411765